2. 임베딩 파일 로드 및 유사도 측정

In [2]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.8 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=7c9591e5b4325cc6a97231e74df1e76d4ae6b3357d8f85fa52cb5e1a75a82186
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [5]:
import pandas as pd
from sentence_transformers import SentenceTransformer

# SentenceTransformer 모델 로드
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

# 엑셀 파일 읽기
file_path = 'Question.xlsx'
df = pd.read_excel('/content/Qusetion.xlsx')

# '질문 (Query)','답변(Answer)' 열의 내용을 가져와서 임베딩 처리
queries = df[['질문 (Query)','답변 (Answer)']].values.tolist()

# 각 질문을 임베딩하여 리스트에 저장
embeddings = model.encode(queries)

# 임베딩된 결과를 새로운 열로 추가
df['embedding_vector'] = embeddings.tolist()

# 수정된 데이터프레임을 새로운 엑셀 파일로 저장
output_file_path = 'Question_with_Embeddings_ST.xlsx'
df.to_excel(output_file_path, index=False)



In [13]:
question= df['질문 (Query)']
question

0                강예인에게 어떻게 연락하면 될까?
1                     강예인에게 연락하고 싶어
2                      강예인의 번호를 알려줘
3                    강예인의 전공을 알고 싶어
4                      강예인의 전공이 뭐야?
                   ...             
321            수학과 복수전공 출신은 누가 있을까?
322          수학과를 복수전공한 사람에는 누가 있어?
323     데이터사이언스학과를 복수전공한 사람은 누가 있어?
324      데이터사이언스학과 복수전공 출신은 누가 있을까?
325    데이터사이언스학과를 복수전공한 사람에는 누가 있어?
Name: 질문 (Query), Length: 326, dtype: object

In [14]:
answer=df['답변 (Answer)']
answer

0      전화번호 : 010-2397-9892, 이메일 : kangyein9892@gmail...
1      전화번호 : 010-2397-9892, 이메일 : kangyein9892@gmail...
2      전화번호 : 010-2397-9892, 이메일 : kangyein9892@gmail...
3                                           제 1전공 : 정보보호
4                                           제 1전공 : 정보보호
                             ...                        
321                                김수민님이 수학과를 복수전공했어요 :)
322                                김수민님이 수학과를 복수전공했어요 :)
323                          이서현님이 데이터사이언스학과를 복수전공했어요 :)
324                          이서현님이 데이터사이언스학과를 복수전공했어요 :)
325                          이서현님이 데이터사이언스학과를 복수전공했어요 :)
Name: 답변 (Answer), Length: 326, dtype: object

In [6]:
#1. 질문열의 공백을 제거하는 코드
df['질문 (Query)'] = df['질문 (Query)'].str.strip()

#2.질문 문장 인코딩 후 텐서
import torch
# 질문 열('질문 (Query)')의 데이터를 가져옴
sentences = df['질문 (Query)'].tolist()

# 문장 인코딩
sentence_embeddings = model.encode(sentences)

# 인코딩된 결과를 PyTorch 텐서로 변환
tensor_embeddings = torch.tensor(sentence_embeddings)

# 텐서 확인
print(tensor_embeddings)

# 3. 코사인 유사도 계산
from sklearn.metrics.pairwise import cosine_similarity
similarities = cosine_similarity(sentence_embeddings, tensor_embeddings)
print(similarities)

tensor([[ 0.1004, -0.2406, -0.8142,  ..., -0.2475, -0.8355, -0.4555],
        [ 0.2152, -0.3781, -1.4457,  ..., -0.1475, -0.3373, -0.7575],
        [-0.1661, -0.2999, -0.3447,  ..., -0.7778, -0.4253, -0.0340],
        ...,
        [ 0.4802, -0.1664, -0.6722,  ...,  1.1298,  0.0243,  0.3059],
        [ 0.4162, -0.0113, -0.6368,  ...,  1.3691,  0.1002, -0.1616],
        [ 0.5886,  0.0427, -0.4040,  ...,  1.0908, -0.0666,  0.1542]])


In [17]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


# 선택된 질문
selected_question = "AI부서에는 누가 있어?"

# 선택된 질문 출력
print(f"선택된 질문: {selected_question}")


# 선택된 질문 문장 인코딩
selected_question_embedding = model.encode([selected_question])

# 저장된 데이터셋의 질문 열의 임베딩 데이터 가져오기
saved_embeddings = df['embedding_vector'].tolist()
tensor_saved_embeddings = torch.tensor(saved_embeddings)

# 선택된 질문과 저장된 데이터셋의 질문 간 코사인 유사도 계산
similarities = cosine_similarity(selected_question_embedding, tensor_saved_embeddings)

# 유사도가 가장 높은 질문의 인덱스 찾기
most_similar_index = similarities.argmax()

# 가장 높은 유사도를 가진 질문의 인덱스의 답변 출력
selected_answer = df.loc[most_similar_index, '답변 (Answer)']
print(f"선택된 질문과 가장 유사한 질문의 답변: {selected_answer}")


선택된 질문: AI부서에는 누가 있어?
선택된 질문과 가장 유사한 질문의 답변: ai/ml 부서에 물어보시면 됩니다, ai/ml 부서에는 다음과 같은 사람들이 있습니다 (배지윤, 이재연, 정다혜, 김수민, 오윤선, 이서현)
